## Motivation

In probability and statistics, standardization is a process that scales and centers variables.
For a scalar random variable $X$, its standardization is the variable $Z$ defined by

$$
\begin{equation}
Z = \frac{X - \mathbb{E} X}{\sqrt{\operatorname{Var}(X)}}.
\end{equation}
$$

Standardization allows for easier comparison and analysis of different variables, providing a common ground for meaningful interpretations.
A good example of this is the use of [standardized coefficients](https://en.wikipedia.org/wiki/Standardized_coefficient) in regression.

If $X$ is instead a random *vector*, the above formula falls short since $\operatorname{Var}(X)$ is a covariance matrix.
In this short exposition, the above formula is generalized to the vector (a.k.a. multivariate) setting.

As an application, we consider a well-known recipe for sampling from multivariate random normal distributions using a standard random normal sampler.

## Prerequisites

* [Independence](https://en.wikipedia.org/wiki/Independence_(probability_theory))
* [Random vector](https://en.wikipedia.org/wiki/Multivariate_random_variable)
* [Covariance matrix](https://en.wikipedia.org/wiki/Covariance_matrix)
* [Positive definite matrix](https://en.wikipedia.org/wiki/Positive-definite_matrix)
* [Cholesky factorization](https://en.wikipedia.org/wiki/Cholesky_decomposition)
* [Multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)

## Standardizing a random vector

Standardization in multiple dimensions is an immediate consequence of the following result:

**Proposition (First and Second Moments of Affine Transform)**.
Let $U$ and $V$ be random vectors, $A$ be a (deterministic) matrix and $b$ be a (deterministic) vector such that

$$
U = A V + b.
$$

Then,

$$
\begin{align*}
\mathbb{E}U & =A\mathbb{E}V+b\\
\operatorname{Var}(U) & =A\operatorname{Var}(V)A^{\intercal}.
\end{align*}
$$

You can prove the above by direct computation, using linearity of expectation and the identity $\operatorname{Var}(Y) = \mathbb{E}[YY^\intercal] - \mathbb{E}Y\mathbb{E}Y^\intercal$.

**Corollary (Standardization)**.
Let $\mu$ be a vector and $\Sigma$ be a positive definite matrix so that it admits a Cholesky factorization $LL^\intercal = \Sigma$.
Let $X$ and $Z$ be random vectors satisfying

$$
\begin{equation}
Z = L^{-1} \left( X - \mu \right).
\end{equation}
$$

Then, $X$ has mean $\mu$ and covariance matrix $\Sigma$ if and only if $Z$ has mean $\mathbf{0}$ and covariance matrix $I$.

Note that in the above, $L$ generalizes $\sqrt{\operatorname{Var}(X)}$ from the scalar case.

### Bivariate case

It is useful, for the sake of reference, to derive closed forms for the Cholesky factorization in the $d = 2$ case.
In this case, the covariance matrix takes the form

$$
\Sigma=\begin{pmatrix}\sigma_{1}\\
 & \sigma_{2}
\end{pmatrix}\begin{pmatrix}1 & \rho\\
\rho & 1
\end{pmatrix}\begin{pmatrix}\sigma_{1}\\
 & \sigma_{2}
\end{pmatrix}.
$$

It is easy to verify (by matrix multiplication) that the Cholesky factorization is given by

$$
L=\begin{pmatrix}\sigma_{1}\\
 & \sigma_{2}
\end{pmatrix}\begin{pmatrix}1 & 0\\
\rho & \sqrt{1-\rho^{2}}
\end{pmatrix}.
$$

Its inverse is

$$
L^{-1}=\begin{pmatrix}1 & 0\\
-\frac{\rho}{\sqrt{1-\rho^{2}}} & \frac{1}{\sqrt{1-\rho^{2}}}
\end{pmatrix}.
$$

# Sampling a multivariate random normal distribution

A [multivariate random normal](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) variable is defined as any random vector $X$ which can be written in the form

$$
X = LZ + \mu
$$

where $Z$ is a random vector whose coordinates are independent standard normal variables.
From our results above, we know that $X$ has mean $\mu$ and covariance $\Sigma = LL^\intercal$.

*Remark*. This only *one* of many equivalent ways to define a multivariate random normal variable.

This gives us a recipe for simulating draws from an arbitrary multivariate random normal distribution given only a standard random normal sampler such as `np.random.randn()`:

In [47]:
import matplotlib.pyplot as plt
import numpy as np

plt.style.use("fivethirtyeight")

In [46]:
def sample_multivariate_normal(
    mean: np.ndarray,
    covariance: np.ndarray,
    n_samples: int = 1,
) -> np.ndarray:
    """Samples a multivariate random normal distribution.
    
    Parameters
    ----------
    mean
        ``(d, )`` shaped mean
    covariance
        Positive definite ``(d, d)`` shaped covariance matrix
    n_samples
        Number of samples

    Returns
    -------
    ``(n_samples, d)`` shaped array where each row corresponds to a single draw from a multivariate normal.
    """
    chol = np.linalg.cholesky(covariance)
    rand = np.random.randn(mean.shape[0], n_samples)
    return (chol @ rand).T + mean

We can verify this is working as desired with a small test and visualization:

In [48]:
mean       = np.array([5., 10.])
covariance = np.array([[2., 1.],
                       [1., 4.]])

samples = sample_multivariate_normal(mean=mean,
                                     covariance=covariance,
                                     n_samples=10_000)

x, y = samples.T
plt.scatter(x, y, alpha=0.1)

empirical_covariance = np.cov(samples, rowvar=False)

print(f"""
Empirical covariance:
{empirical_covariance}

True covariance:
{covariance}
""")


Empirical covariance:
[[1.96748266 1.02186592]
 [1.02186592 3.98317796]]

True covariance:
[[2. 1.]
 [1. 4.]]



## Deriving the density of a multivariate random normal distribution

Let

$$
X = LZ + \mu
$$

be a multivariate random normal variable.

First, note that $L^{-1}$ is a nonnegative matrix and hence

$$
\mathbb{P}(X\leq x)=\mathbb{P}(LZ+\mu\leq x)=\mathbb{P}(Z\leq L^{-1}\left(x-\mu\right)).
$$

Moreover,

$$
\begin{align*}
\mathbb{P}(X\leq x) & =\int_{-\infty}^{L^{-1}(x-\mu)}\frac{\exp\left(-\frac{1}{2}\left\Vert z\right\Vert ^{2}\right)}{\sqrt{\left(2\pi\right)^{d}}}dz\\
 & =\int_{-\infty}^{x}\frac{\exp\left(-\frac{1}{2}\left\Vert L^{-1}\left(z-\mu\right)\right\Vert ^{2}\right)}{\sqrt{\left(2\pi\right)^{d}}}\det(L^{-1})dz\\
 & =\int_{-\infty}^{x}\frac{\exp\left(-\frac{1}{2}\left(z-\mu\right)^{\intercal}\Sigma^{-1}\left(z-\mu\right)\right)}{\sqrt{\left(2\pi\right)^{d}\det(\Sigma)}}dz
\end{align*}
$$

where we have, in the last equality, used the fact

$$
\det(L^{-1})^{2}=\frac{1}{\det(L)^{2}}=\frac{1}{\det(LL^{\intercal})}=\frac{1}{\det(\Sigma)}.
$$

Taking the gradient with respect to $x$ yields the density

$$
f_{X}(x)=\frac{\exp\left\{ -\frac{1}{2}\left(\left(x-\mu\right)^{\intercal}\Sigma\left(x-\mu\right)\right)\right\} }{\sqrt{\left(2\pi\right)^{d}\det(\Sigma)}}.
$$